In [52]:
import csv

header = ['timer', 'fight_result', 'has_round_started', 'is_round_over', 'Player1_ID', 'health', 'x_coord', 'y_coord', 'is_jumping', 'is_crouching', 'is_player_in_move', 'move_id', 'player1_buttons up', 'player1_buttons down', 'player1_buttons right', 'player1_buttons left', 'player1_button_Y' , 'player1_button_B' , 'player1_button_A' , 'player1_button_X' , 'player1_button_L' , 'player1_button_R' , 'Player2_ID', 'Player2 health', 'Player2 x_coord', 'Player2 y_coord', 'Player2 is_jumping', 'Player2 is_crouching', 'Player2 is_player_in_move', 'Player2 move_id', 'player2_buttons up', 'player2_buttons down', 'player2_buttons right', 'player2_buttons left' , 'player2_button_Y' , 'player2_button_B' , 'player2_button_A' , 'player2_button_X' , 'player2_button_L' , 'player2_button_R']

with open('player1_dataset.csv', 'r+', newline='') as file:
    # Read the existing content
    content = file.read()
    # Move the file pointer to the beginning of the file
    file.seek(0, 0)
    # Add the header row at the beginning of the file
    writer = csv.writer(file)
    writer.writerow(header)
    # Add the existing content after the header row
    file.write(content)


with open('player2_dataset.csv', 'r+', newline='') as file:
    # Read the existing content
    content = file.read()
    # Move the file pointer to the beginning of the file
    file.seek(0, 0)
    # Add the header row at the beginning of the file
    writer = csv.writer(file)
    writer.writerow(header)
    # Add the existing content after the header row
    file.write(content)

In [73]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import pickle



player1_data = pd.read_csv('player1_dataset.csv')   
player1_data = player1_data.drop(['Player1_ID', 'Player2_ID' , 'timer', 'is_round_over','fight_result'], axis=1)
player1_data = player1_data[player1_data['has_round_started']]
player1_data = player1_data.drop(['has_round_started'], axis=1)

player2_data = pd.read_csv('player2_dataset.csv')
player2_data = player2_data.drop(['Player1_ID','Player2_ID', 'timer', 'is_round_over','fight_result'], axis=1)
player2_data = player2_data[player2_data['has_round_started']]
player2_data = player2_data.drop(['has_round_started'], axis=1)


player1_data = player1_data.replace(True, 1)
player1_data = player1_data.replace(False, 0)
player2_data = player2_data.replace(True, 1)
player2_data = player2_data.replace(False, 0)


X1 = player1_data[['health', 'x_coord' , 'y_coord' ,'is_jumping', 'is_crouching', 'player1_buttons up', 'player1_buttons down', 'player1_buttons right', 'player1_buttons left', 'player1_button_Y', 'player1_button_B', 'player1_button_A', 'player1_button_X', 'player1_button_L', 'player1_button_R', 'Player2 health' , 'Player2 x_coord', 'Player2 y_coord', 'Player2 is_jumping', 'Player2 is_crouching', 'player2_buttons up', 'player2_buttons down', 'player2_buttons right', 'player2_buttons left', 'player2_button_Y', 'player2_button_B', 'player2_button_A', 'player2_button_X', 'player2_button_L', 'player2_button_R']]
Y1 = player1_data[['player1_buttons up', 'player1_buttons down', 'player1_buttons right', 'player1_buttons left', 'player1_button_Y', 'player1_button_B', 'player1_button_A', 'player1_button_X', 'player1_button_L', 'player1_button_R']]
X2 = player2_data[['health', 'x_coord' , 'y_coord' ,'is_jumping', 'is_crouching', 'player1_buttons up', 'player1_buttons down', 'player1_buttons right', 'player1_buttons left', 'player1_button_Y', 'player1_button_B', 'player1_button_A', 'player1_button_X', 'player1_button_L', 'player1_button_R', 'Player2 health' , 'Player2 x_coord', 'Player2 y_coord', 'Player2 is_jumping', 'Player2 is_crouching', 'player2_buttons up', 'player2_buttons down', 'player2_buttons right', 'player2_buttons left', 'player2_button_Y', 'player2_button_B', 'player2_button_A', 'player2_button_X', 'player2_button_L', 'player2_button_R']]
Y2 = player2_data[['player2_buttons up', 'player2_buttons down', 'player2_buttons right', 'player2_buttons left', 'player2_button_Y', 'player2_button_B', 'player2_button_A', 'player2_button_X', 'player2_button_L', 'player2_button_R']]


X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.3, random_state=0)
X_train1, X_test1, y_train1, y_test1 = train_test_split(X2, Y2, test_size=0.3, random_state=0)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
Y_train = scaler.fit_transform(y_train)
Y_test = scaler.transform(y_test)


X_train1 = scaler.fit_transform(X_train1)
X_test1 = scaler.transform(X_test1)
Y_train1 = scaler.fit_transform(y_train1)
Y_test1 = scaler.transform(y_test1)


model1 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(Y_train.shape[1])
])

model2 = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_shape=(X_train1.shape[1],), activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(Y_train1.shape[1])
])


model1.compile(optimizer='adam', loss='mse', metrics=['accuracy'])


model2.compile(optimizer='adam', loss='mse', metrics=['accuracy'])


history1 = model1.fit(X_train, Y_train, epochs=200, batch_size=64, validation_data=(X_test, Y_test))


history2 = model2.fit(X_train1, Y_train1, epochs=200, batch_size=64, validation_data=(X_test1, Y_test1))


test_loss, test_acc = model1.evaluate(X_test, Y_test, verbose=2)
print('Test accuracy:', test_acc)

print()

print()

#test_loss1 = model2.evaluate(X_test1, Y_test1)
test_loss1, test_acc1 = model2.evaluate(X_test1, Y_test1, verbose=2)
print('Test accuracy:', test_acc1)


with open('model1.pkl', 'wb') as f:
    pickle.dump(model1, f)
    
with open('model2.pkl', 'wb') as f:
    pickle.dump(model2, f)



Epoch 1/200
49/49 [==============================] - 3s 11ms/step - loss: 0.4108 - accuracy: 0.2960 - val_loss: 0.1885 - val_accuracy: 0.3721
Epoch 2/200
49/49 [==============================] - 0s 6ms/step - loss: 0.0780 - accuracy: 0.4185 - val_loss: 0.0462 - val_accuracy: 0.5116
Epoch 3/200
49/49 [==============================] - 0s 6ms/step - loss: 0.0282 - accuracy: 0.5042 - val_loss: 0.0237 - val_accuracy: 0.5429
Epoch 4/200
49/49 [==============================] - 0s 6ms/step - loss: 0.0167 - accuracy: 0.5480 - val_loss: 0.0172 - val_accuracy: 0.5421
Epoch 5/200
49/49 [==============================] - 0s 6ms/step - loss: 0.0118 - accuracy: 0.5761 - val_loss: 0.0130 - val_accuracy: 0.5503
Epoch 6/200
49/49 [==============================] - 0s 6ms/step - loss: 0.0089 - accuracy: 0.5892 - val_loss: 0.0106 - val_accuracy: 0.5697
Epoch 7/200
49/49 [==============================] - 0s 9ms/step - loss: 0.0071 - accuracy: 0.5870 - val_loss: 0.0089 - val_accuracy: 0.6018
Epoch 8/200
